In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [48]:
df["previous_content_id"] = df.groupby("user_id")["content_id"].shift(1)
df["previous_answered_correctly"] = df.groupby("user_id")["answered_correctly"].shift(1)
df["previous_user_answer"] = df.groupby("user_id")["user_answer"].shift(1)

In [49]:
df["content_id"].value_counts().head(30)

6116     21280
6173     20076
4120     19898
175      19418
7876     18885
7900     17977
2063     17423
2064     17423
2065     17423
4492     17304
4696     17151
3363     17044
3365     17044
3364     17044
1278     16979
6370     16897
2946     16619
2948     16619
2947     16619
2594     15992
2593     15992
2595     15991
7218     15900
7217     15900
7216     15900
7219     15899
10686    13618
10688    13364
294      13152
10685    12772
Name: content_id, dtype: int64

In [68]:
df[df["content_id"]==6116]["previous_content_id"].value_counts()

4696.0     14456
6372.0      1013
29579.0      247
4108.0       126
4120.0        99
           ...  
7090.0         1
4589.0         1
593.0          1
8969.0         1
8959.0         1
Name: previous_content_id, Length: 2562, dtype: int64

In [59]:
df_oof = pd.read_csv("../output/ex_172/20201202080625/oof_train_0_lgbm.csv")
df_oof = df_oof.set_index("row_id")
df_oof = df_oof.reset_index()
df_oof.columns = ["index", "predict", "target"]

In [60]:
df_oof = pd.merge(df_oof, df.reset_index(), how="inner")

In [69]:
now = 6116
prev = 4696

In [70]:
df[(df["content_id"] == now) & (df["previous_content_id"] == prev)].groupby(["previous_answered_correctly"])["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
previous_answered_correctly,,,,,,,,
0.0,6592.0,0.165049,0.371253,0.0,0.0,0.0,0.0,1.0
1.0,7864.0,0.204858,0.403623,0.0,0.0,0.0,0.0,1.0


In [71]:
df_oof[(df_oof["content_id"] == now) & (df_oof["previous_content_id"] == prev)].groupby(["previous_answered_correctly"])[["answered_correctly", "predict"]].describe()

answered_correctly                                \
                                         count      mean       std  min  25%   
previous_answered_correctly                                                    
0.0                                      741.0  0.164642  0.371108  0.0  0.0   
1.0                                      826.0  0.199758  0.400061  0.0  0.0   

                                           predict                      \
                             50%  75%  max   count      mean       std   
previous_answered_correctly                                              
0.0                          0.0  0.0  1.0   741.0  0.161223  0.058282   
1.0                          0.0  0.0  1.0   826.0  0.197080  0.101678   

                                                                               
                                  min       25%       50%       75%       max  
previous_answered_correctly                                                    
0.0                          0.090047  0.120965  0.147494  0.178781  0.744017  
1.0                          0.093129  0.132884  0.164520  0.223584  0.865419

In [72]:
df[(df["content_id"] == now) & (df["previous_content_id"] == prev)].groupby(["previous_user_answer"])["answered_correctly"].describe()

,count,mean,std,min,25%,50%,75%,max
previous_user_answer,,,,,,,,
0.0,7864.0,0.204858,0.403623,0.0,0.0,0.0,0.0,1.0
1.0,2566.0,0.162510,0.368990,0.0,0.0,0.0,0.0,1.0
2.0,1947.0,0.151002,0.358142,0.0,0.0,0.0,0.0,1.0
3.0,2079.0,0.181337,0.385390,0.0,0.0,0.0,0.0,1.0


In [73]:
df_oof[(df_oof["content_id"] == now) & (df_oof["previous_content_id"] == prev)].groupby(["previous_user_answer"])[["answered_correctly", "predict"]].describe()

answered_correctly                                     \
                                  count      mean       std  min  25%  50%   
previous_user_answer                                                         
0.0                               826.0  0.199758  0.400061  0.0  0.0  0.0   
1.0                               282.0  0.148936  0.356658  0.0  0.0  0.0   
2.0                               223.0  0.147982  0.355881  0.0  0.0  0.0   
3.0                               236.0  0.199153  0.400212  0.0  0.0  0.0   

                               predict                                \
                      75%  max   count      mean       std       min   
previous_user_answer                                                   
0.0                   0.0  1.0   826.0  0.197080  0.101678  0.093129   
1.0                   0.0  1.0   282.0  0.162449  0.054402  0.090047   
2.0                   0.0  1.0   223.0  0.158949  0.066675  0.093263   
3.0                   0.0  1.0   236.0  0.161908  0.054318  0.095084   

                                                              
                           25%       50%       75%       max  
previous_user_answer                                          
0.0                   0.132884  0.164520  0.223584  0.865419  
1.0                   0.120844  0.146504  0.183349  0.362410  
2.0                   0.120514  0.142374  0.172585  0.744017  
3.0                   0.121957  0.150471  0.178705  0.412380